In [1]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

import utils

Define constants like location and the current day-of-year for analysis.

In [2]:
# Build arrays of dates
dates = pd.date_range("2024-01-01", "2024-12-31")
doys = dates.day_of_year

# August 23
today = 235
latitude = 40.5
target_day_length = utils.get_day_length(latitude, today)
print(f"{target_day_length:.2f} hours of daylight on {dates[today]:%B %d} at latitude ~{latitude}°.")

13.29 hours of daylight on August 23 at latitude ~40.5°.


## Travel Stats

Plot daylight hours by day at the given location.

In [3]:
px.line(
    x=dates, 
    y=[utils.get_day_length(latitude=latitude, doy=doy) for doy in doys],
    labels={"x": "Date", "y": "Daylight hours"},
    title=f"Daylight hours at ~{latitude}° Latitude",
    width=600,
)

Plot latitude needed to maintain the same number of daylight hours.

In [4]:
latitudes = [utils.get_latitude(day_length=target_day_length, doy=doy) for doy in doys]

px.line(
    x=dates, 
    y=latitudes,
    labels={"x": "Date", "y": "Latitude"},
    title=f"Latitude with {target_day_length:.2f} daylight hours",
    width=600,

)

Plot distance traveled per day. Couple of long travel days on the equinoxes...

In [5]:
px.line(
    x=doys, 
    y=[utils.haversine_distance(latitudes[i], latitudes[i - 1]) for i in range(len(doys))],
    labels={"x": "Date", "y": "Kilometers traveled"},
    title=f"Travel distance to keep {target_day_length:.2f} daylight hours",
    width=600,
)

Calculate mean velocity each month, assuming you only spend 8 hours traveling per day.

In [6]:
speed_df = pd.DataFrame([utils.haversine_distance(latitudes[i], latitudes[i - 1]) / 8.0 for i in range(len(doys))], index=dates)
speed_df.groupby(by=speed_df.index.month_name()).mean()

,0
April,16.407419
August,10.475359
December,0.773043
February,10.148011
January,2.685984
July,2.649399
June,0.784001
March,106.996191
May,4.410120
November,4.204325


Calculate mean velocity in March before the equinox.

In [7]:
speed_df[(speed_df.index >= "2024-03-01") & (speed_df.index < "2024-03-22")].mean()

0    26.916137
dtype: float64

Calculate velocity *on* the vernal equinox.

In [8]:
speed_df[speed_df.index == "2024-03-22"]

,0
2024-03-22,2468.504392


## Daylight Hour Grid

In [9]:
# This is hacky, but this version of the df uses intervals of 5 degrees and 5 days to 
# allow plotting, while the other uses intervals of 1 degree and 1 day to plot matching days
day_length_by_latitude_coarse = pd.DataFrame([
    {"latitude": lat, "date": date, "doy": date.day_of_year, "day_length": utils.get_day_length(latitude=lat, doy=date.day_of_year)}
for lat in range(-90, 91, 5) for date in dates[::5]])


day_length_by_latitude = pd.DataFrame([
    {"latitude": lat, "date": date, "doy": date.day_of_year, "day_length": utils.get_day_length(latitude=lat, doy=date.day_of_year)}
for lat in range(-90, 91) for date in dates])

In [10]:
fig = px.scatter(
    day_length_by_latitude_coarse,
    x="date",
    y="latitude",
    color="day_length",
    color_continuous_scale="viridis",
    labels={"latitude": "Latitude", "date": "Date", "day_length": "Daylight hours"},
)

fig.update_traces(marker=dict(size=10, symbol="square"))

fig.update_layout(
    width=1400, 
    height=600,
    xaxis=dict(tickformat="%b"),
    yaxis=dict(dtick=30),
    paper_bgcolor="rgba(255,255,255,1)",
    plot_bgcolor="rgba(255,255,255,1)",
    font=dict(color="black", size=32),
    margin=dict(t=20, b=50)
)

fig.update_xaxes(
    title=None, 
    showline=False,
    showgrid=False,
)

fig.update_yaxes(
    showline=False,
    showgrid=False,
    zeroline=False,
)

day_length = target_day_length
tolerance = 1

# Get matching days, but avoid drawing days to the right of the grid
matching_days = (
    day_length_by_latitude[day_length_by_latitude.day_length.between(day_length - tolerance/2, day_length + tolerance/2) & day_length_by_latitude.date.le(day_length_by_latitude_coarse.date.max())]
)

fig.add_trace(go.Scattergl(
    x=matching_days.date,
    y=matching_days.latitude,
    mode="markers",
    showlegend=False,
    marker=dict(color="rgba(255, 90, 70, 1.0)")
))

fig

In [11]:
fig = px.scatter(
    day_length_by_latitude_coarse,
    x="date",
    y="latitude",
    color="day_length",
    color_continuous_scale="viridis",
    labels={"latitude": "Latitude", "date": "Date", "day_length": "Daylight hours"},
)

fig.update_traces(marker=dict(size=11, symbol="square"))

fig.update_layout(
    width=1600, 
    height=600,
    xaxis=dict(tickformat="%b"),
    yaxis=dict(dtick=30),
    paper_bgcolor="rgba(255,255,255,1)",
    plot_bgcolor="rgba(255,255,255,1)",
    font=dict(color="black", size=36),
    margin=dict(t=20, b=50)
)

fig.update_xaxes(
    title=None, 
    showline=False,
    showgrid=False,
)

fig.update_yaxes(
    showline=False,
    showgrid=False,
    zeroline=False,
)

day_length = target_day_length
tolerance = 1

# Get matching days, but avoid drawing days to the right of the grid
matching_days = (
    day_length_by_latitude[day_length_by_latitude.day_length.between(day_length - tolerance/2, day_length + tolerance/2) & day_length_by_latitude.date.le(day_length_by_latitude_coarse.date.max())]
)

fig.add_trace(go.Scattergl(
    x=matching_days.date,
    y=matching_days.latitude,
    mode="markers",
    showlegend=False,
    marker=dict(color="rgba(255, 90, 70, 1.0)")
))

fig

In [12]:
fig.write_image("../figures/day_length_13p5hr.png", scale=0.5)